In [6]:
import pandas as pd
import sqlite3 as sql

pd.set_option('display.max_colwidth', None)

In [7]:
connection = sql.connect('data/sql-murder-mystery.db')

crsr = connection.cursor()


In [8]:
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

In [9]:
sql_query('Select * from crime_scene_report where date=20180115 and city="SQL City" and type="murder"')

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [10]:
sql_query('Select * from person where name like "%Annabel%" and address_street_name="Franklin Ave"')

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [11]:
sql_query('Select * from person where address_street_name="Northwestern Dr" and address_number=(Select max(address_number) from person where address_street_name="Northwestern Dr")')

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [12]:
sql_query('Select * from interview where person_id=16371 or person_id=14887')

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [13]:
sql_query('Select * from get_fit_now_member, drivers_license, get_fit_now_check_in, person where get_fit_now_check_in.check_in_date=20180109 and get_fit_now_member.id like "%48Z%" and get_fit_now_member.membership_status="gold" and drivers_license.plate_number like "%H42W%" and drivers_license.gender="male" and get_fit_now_check_in.membership_id=get_fit_now_member.id and get_fit_now_member.person_id=person.id and person.license_id=drivers_license.id')


,id,person_id,name,membership_start_date,membership_status,id,age,height,eye_color,hair_color,...,membership_id,check_in_date,check_in_time,check_out_time,id,name,license_id,address_number,address_street_name,ssn
0,48Z55,67318,Jeremy Bowers,20160101,gold,423327,30,70,brown,brown,...,48Z55,20180109,1530,1700,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [14]:
#El asesino es Jeremy Browers con person id 67318